In [1]:
# TO FIX:
# add periodtype = semi
# add periodtype = quart
# fix final balance to equal 0
# might want to account for leap years somehow (I think this is already accounted for)


from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pandas as pd
import re
import calendar
from statistics import mean

# function created for this project (should create link if I ever share this)
from validate_date import validate_date

def loan_amortization(principal, interest_rate, start_date, term_years, periodtype):
    
    """ 
        Args:
        principal (int or float): The principal amount of the loan.
        interest_rate (int or float): The annual interest rate, expressed as a percent (e.g. 5% = 5).
        start_date (str or datetime): The start date of the loan in 'YYYY-MM-DD' format or as a datetime object.
        term_years (int or float): Loan/Borrowing term in years.
        periodtype (str): The type of period to use for the loan payments, which can be one of the following:
            'D' (days), 'W' (weeks), 'BW' (biweekly), 'M-30' (months 30/360), 'M-Actual' (months days_in_month/360), or 'Y' (years).

    Returns:
        pandas.DataFrame: A DataFrame containing the amortization schedule for the loan, with columns for the payment number, payment date, payment amount, interest paid, principal paid, and balance.
    """
    if not isinstance(term_years, (int, float)):
        raise TypeError("term_years should be numeric (int or float)")
    
    if validate_date(start_date) == False:
        raise TypeError("start_date must be a string in 'YYYY-MM-DD' format or a datetime object")
    
    # if the date is in the string format, convert it
    if not isinstance(start_date, datetime):
        start_date = datetime.strptime(start_date, '%Y-%m-%d')
    
    # create end date of term using term_years
    end_date = start_date + timedelta(days=int(365.25 * term_years))
    
    # creating list of dates for index in months
    dates = [start_date]
    current_date = start_date
    while current_date < end_date:
        current_date += relativedelta(months=1)
        dates.append(current_date)
    dates.pop(-1)
    
    # create list of days in the month of each date
    days_in_month = [calendar.monthrange(date.year, date.month)[1] for date in dates]
                        
    # input type checking for principal and interest_rate
    if not isinstance(principal, (int, float)):
        raise TypeError("Principal amount should be numeric (int or float)")
    if not isinstance(interest_rate, (int, float)):
        raise TypeError("Interest rate should be numeric and in % (int or float)")
        
    # period-type definition 
    if periodtype == 'D':
        periods = int((end_date - start_date).days)
        days_between_payments = 1
        adjusted_rate = interest_rate / 36525
    elif periodtype == 'W':
        periods = int((end_date - start_date).days / 7)
        days_between_payments = 7
        adjusted_rate = interest_rate / 5200
    elif periodtype == 'BW':
        periods = int((end_date - start_date).days / 14)
        days_between_payments = 14
        adjusted_rate = interest_rate / 2600
    elif periodtype == 'M-30':
        if start_date.month < end_date.month:
            periods = int((end_date.year - start_date.year) * 12 + (end_date.month - start_date.month) + 1)
        elif start_date.month > end_date.month:
            periods = int((end_date.year - start_date.year) * 12 + (end_date.month - start_date.month) - 1)
        else:
            periods = int((end_date.year - start_date.year) * 12)
        days_between_payments = 30
        adjusted_rate = interest_rate / 1200
    elif periodtype == 'M-Actual':
        if start_date.month < end_date.month:
            periods = int((end_date.year - start_date.year) * 12 + (end_date.month - start_date.month) + 1)
        elif start_date.month > end_date.month:
            periods = int((end_date.year - start_date.year) * 12 + (end_date.month - start_date.month) - 1)
        else:
            periods = int((end_date.year - start_date.year) * 12)
        monthly_rate = [interest_rate / 36000 * days_in_month[i] for i in range(len(dates))]
        adjusted_rate = mean(monthly_rate)
    elif periodtype == 'Y':
        periods = int(end_date.year - start_date.year)
        adjusted_rate = interest_rate / 100
    else:
        raise TypeError("periodtype should be one of the following: 'D', 'W', 'BW', 'M-30', 'M-Actual', 'Y'")

    monthly_payment = principal * adjusted_rate / (1 - (1 + adjusted_rate) ** (-periods))

    # create a list of dates for each payment
    if periodtype == 'M-Actual' or periodtype == 'M-30':
        payment_dates = dates
    elif periodtype == 'Y':
        payment_dates = [start_date + relativedelta(years=1 * i) for i in range(periods)]
    else:
        payment_dates = [start_date + relativedelta(days=(days_between_payments * i)) for i in range(periods)]

    # lists for the payment number, payment amount, interest, principal, and balance
    payment_number = list(range(1, periods + 1))
    payment_amount = monthly_payment
    interest = []
    principal_paid = []
    beg_balance = [principal]
    end_balance = []

    # interest, principal, and balance for each payment period (exlcuding M-actual)
    if not periodtype == "M-Actual":
        for i in range(periods):
            interest.append(beg_balance[i] * adjusted_rate)
            principal_paid.append(monthly_payment - interest[i])
            beg_balance.append(beg_balance[i] - principal_paid[i])
            end_balance.append(beg_balance[i] - principal_paid[i - 1])
    elif periodtype == "M-Actual":
        for i in range(periods):
            interest.append(beg_balance[i] * monthly_rate[i])
            principal_paid.append(monthly_payment - interest[i])
            beg_balance.append(beg_balance[i] - principal_paid[i])
            end_balance.append(beg_balance[i] - principal_paid)
            
#     elif periodtype == "Semi":
        
#     elif periodtype == "Quart"
        
  
    # make the amortization-schedule dataframe
    data = {
        'Payment Number': payment_number,
        'Payment Date': payment_dates,
        'Beginning Balance': beg_balance[:-1],
        'Payment Amount': payment_amount,
        'Interest Paid': interest,
        'Principal Paid': principal_paid,
        'Ending Balance': end_balance
    }
    df = pd.DataFrame(data)

    # set the index to the payment date
    df.set_index('Payment Date', inplace=True)


    # # Truncate the DataFrame to the end date
    # df = df.truncate(before=start_date, after=end_date)

    # apply formating for dollar signs and two decimals
    df.index.name = 'Date'
    df['Payment Amount'] = df['Payment Amount'].apply(lambda x: '${:,.2f}'.format(x))
    df['Interest Paid'] = df['Interest Paid'].apply(lambda x: '${:,.2f}'.format(x))
    df['Principal Paid'] = df['Principal Paid'].apply(lambda x: '${:,.2f}'.format(x))
    df['Beginning Balance'] = df['Beginning Balance'].apply(lambda x: '${:,.2f}'.format(x))
    df['Ending Balance'] = df['Ending Balance'].apply(lambda x: '${:,.2f}'.format(x))

    return df

In [2]:
# Test: 5-year loan, starting on January 1, 2023 and ending on December 31, 2027
schedule = loan_amortization(10000, 5, "2023-1-1", 10, 'W')

schedule

,Payment Number,Beginning Balance,Payment Amount,Interest Paid,Principal Paid,Ending Balance
Date,,,,,,
2023-01-01,1,"$10,000.00",$24.41,$9.62,$14.79,"$9,985.21"
2023-01-08,2,"$9,985.21",$24.41,$9.60,$14.81,"$9,970.41"
2023-01-15,3,"$9,970.40",$24.41,$9.59,$14.82,"$9,955.59"
2023-01-22,4,"$9,955.57",$24.41,$9.57,$14.84,"$9,940.75"
2023-01-29,5,"$9,940.73",$24.41,$9.56,$14.85,"$9,925.90"
...,...,...,...,...,...,...
2032-11-21,517,$121.70,$24.41,$0.12,$24.29,$97.43
2032-11-28,518,$97.41,$24.41,$0.09,$24.32,$73.11
2032-12-05,519,$73.09,$24.41,$0.07,$24.34,$48.77


In [3]:
schedule = loan_amortization(10000, 5, "2023-1-1", 10, 'M-30')

schedule

,Payment Number,Beginning Balance,Payment Amount,Interest Paid,Principal Paid,Ending Balance
Date,,,,,,
2023-01-01,1,"$10,000.00",$106.07,$41.67,$64.40,"$9,935.60"
2023-02-01,2,"$9,935.60",$106.07,$41.40,$64.67,"$9,871.20"
2023-03-01,3,"$9,870.93",$106.07,$41.13,$64.94,"$9,806.27"
2023-04-01,4,"$9,806.00",$106.07,$40.86,$65.21,"$9,741.06"
2023-05-01,5,"$9,740.79",$106.07,$40.59,$65.48,"$9,675.58"
...,...,...,...,...,...,...
2032-08-01,116,$523.76,$106.07,$2.18,$103.88,$420.31
2032-09-01,117,$419.88,$106.07,$1.75,$104.32,$316.00
2032-10-01,118,$315.56,$106.07,$1.31,$104.75,$211.25


In [4]:
schedule = loan_amortization(10000, 5, "2023-1-1", 10, 'M-Actual')

schedule

,Payment Number,Beginning Balance,Payment Amount,Interest Paid,Principal Paid,Ending Balance
Date,,,,,,
2023-01-01,1,"$10,000.00",$106.43,$43.06,$63.37,"$9,936.63"
2023-02-01,2,"$9,936.63",$106.43,$38.64,$67.78,"$10,000.00"
2023-03-01,3,"$9,868.85",$106.43,$42.49,$63.93,"$9,936.63"
2023-04-01,4,"$9,804.91",$106.43,$40.85,$65.57,"$9,868.85"
2023-05-01,5,"$9,739.34",$106.43,$41.93,$64.49,"$9,804.91"
...,...,...,...,...,...,...
2032-08-01,116,$523.57,$106.43,$2.25,$104.17,$627.30
2032-09-01,117,$419.40,$106.43,$1.75,$104.68,$523.57
2032-10-01,118,$314.72,$106.43,$1.36,$105.07,$419.40


In [5]:
start_date = datetime(2022, 1, 15)
term_years = 30
end_date = start_date + timedelta(days=int(365.25 * term_years))
test = int(end_date.year - start_date.year)
# for i in range(test):
#     print(i)
(end_date - start_date).days

10957

In [6]:
start_date = datetime(2022, 1, 15)
end_date = datetime(2022, 5, 15)

num_days_per_month = []
while start_date <= end_date:
    days_in_month = (start_date + relativedelta(day=1, months=+1, days=-1)).day
    start_date = start_date + relativedelta(months=+1)
    num_days_per_month.append(days_in_month)

print(num_days_per_month)

[31, 28, 31, 30, 31]


In [7]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
from statistics import mean

start_date = datetime(2023, 3, 15)
term_years = 5

end_date = start_date + relativedelta(years=term_years)

dates = [start_date]
current_date = start_date

while current_date < end_date:
    current_date += relativedelta(months=1)  # add one month to current date
    dates.append(current_date)  # append new date to list of dates
    
dates.pop(-1)

days_in_month = [calendar.monthrange(date.year, date.month)[1] for date in dates]


days_in_month[:12]


[31, 30, 31, 30, 31, 31, 30, 31, 30, 31, 31, 29]

In [8]:
interest_rate = 5

monthly_rate = [interest_rate / 36000 * days_in_month[i] for i in range(len(dates))]
mean(monthly_rate)


0.004229166666666667